In [23]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [24]:
# Definir las categorías
categories = ['auto', 'casa', 'edificio', 'gato', 'moto']

In [25]:
# Definir las dimensiones de las imagenes
img_height = 224
img_width = 224

In [26]:
# Creación de un diccionario para mapear los nombres de las imagenes a las categorias
category_map = {}
for img_name in os.listdir('dataset/train'):
    for category in categories:
        if category in img_name:
            category_map[img_name] = categories.index(category)
            break

In [27]:
# Creación de un diccionario para mapear los nombres de las imagenes a las categorias
category_map2 = {}
for img_name in os.listdir('dataset/test'):
    for category in categories:
        if category in img_name:
            category_map2[img_name] = categories.index(category)
            break

In [28]:
# Carga de las imagenes de entrenamiento
X_train = []
y_train = []
for img_name in os.listdir('dataset/train'):
    img = load_img(os.path.join('dataset/train', img_name), target_size=(img_height, img_width))
    img_array = img_to_array(img)
    X_train.append(img_array)
    y_train.append(category_map[img_name])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [29]:
y_train = to_categorical(y_train, num_classes=len(categories))

In [30]:
# Carga de las imagenes de test
X_test = []
y_test = []
for img_name in os.listdir('dataset/test'):
    img = load_img(os.path.join('dataset/test', img_name), target_size=(img_height, img_width))
    img_array = img_to_array(img)
    X_test.append(img_array)
    y_test.append(category_map2[img_name])

X_test = np.array(X_test)
y_test = np.array(y_test)

In [31]:
y_test = to_categorical(y_test, num_classes=len(categories))

In [32]:
# Definir la forma de entrada
input_shape = (img_height, img_width, 3)

In [33]:
# Definir la capa de entrada
input_layer = Input(shape=input_shape)

In [34]:
# Definir las capas convolucionales
x = Conv2D(512, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

In [35]:
# Definir la capa Flatten
x = Flatten()(x)

In [36]:
# Definición de las capas densas
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(len(categories), activation='softmax')(x)

In [37]:
# Definir la capa de salida
output_layer = x

In [38]:
# Creación del modelo
model = Model(inputs=input_layer, outputs=output_layer)

In [39]:
# Compilación del modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [40]:
# Entrenamiento del modelo
ajuste = %time model.fit(X_train, y_train, epochs=160, batch_size=32, validation_split=0.2)
ajuste

Epoch 1/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.2500 - loss: 3.3821 - val_accuracy: 0.0000e+00 - val_loss: 182.5314
Epoch 2/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.2500 - loss: 151.2140 - val_accuracy: 0.0000e+00 - val_loss: 35.1597
Epoch 3/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.2500 - loss: 35.1459 - val_accuracy: 0.0000e+00 - val_loss: 24.1240
Epoch 4/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - accuracy: 0.2500 - loss: 13.4877 - val_accuracy: 0.0000e+00 - val_loss: 9.9124
Epoch 5/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.2500 - loss: 4.6563 - val_accuracy: 0.0000e+00 - val_loss: 6.8356
Epoch 6/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.2500 - loss: 2.3576 - val_accuracy: 0.0000e+00 - val_loss: 5.3678
Epoch 7/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.2857 - loss: 1.4106 - val_accuracy: 0.0000e+00 - val_loss: 5.4126
Epoch 8/160
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.2500 - loss: 1.358

In [41]:
# Evaluar el modelo con datos de test para obtener información aproximada en cuanto al funcionamiento del modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.2f}')
print(f'Test loss: {loss:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6000 - loss: 14.8043
Test accuracy: 0.60
Test loss: 14.80


In [57]:
# Guardar el modelo entrenado en un archivo .h5
model.save('mi_modelo.h5')

In [62]:
# Convertir el modelo Keras guardado a ONNX utilizando td2onnx
spec = (tf.TensorSpec((None, img_height, img_width, 3), tf.float32, name="input"),)
output_path = "mi_modelo.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())

print(f'Model saved to {output_path}')

Model saved to mi_modelo.onnx
